In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as lr
import statsmodels.api as sm


### Homework 1.1 Q1 - Q3

For questions 1 to 3:

Perform a linear regression to predict Y from X1, X2, and X3. Use the file homework_1.1.csv.

In [2]:
# 1. Import Libraries and Load Data
import pandas as pd
import statsmodels.api as sm

# Load the CSV file into the 'df_1_1' DataFrame.
# Ensure the file is in the same directory as your notebook.
try:
    df_1_1 = pd.read_csv('homework_1.1.csv')
    print("File loaded successfully. Data preview:")
    print(df_1_1.head())
except FileNotFoundError:
    print("Error: 'homework_1.1.csv' file not found.")
    df_1_1 = None

File loaded successfully. Data preview:
   Unnamed: 0        X1        X2        X3          Y
0           0 -0.440646 -0.390227  0.156718  -0.877671
1           1 -3.810099 -1.304665 -1.105117 -10.130388
2           2 -1.425451 -0.340049  1.115908   0.284068
3           3 -1.325750  0.161906 -0.254670  -1.994344
4           4  3.120263  1.487343 -1.164839   2.030030


In [3]:
# 2. Perform Multiple Linear Regression
if df_1_1 is not None:
    # Define the dependent (Y) and independent (X1, X2, X3) variables.
    Y = df_1_1['Y']
    X_all = df_1_1[['X1', 'X2', 'X3']]
    
    # Add a constant (intercept) to the model.
    X_all = sm.add_constant(X_all)

    # Fit the Ordinary Least Squares (OLS) model.
    model_all = sm.OLS(Y, X_all).fit()

    # Print the full regression summary.
    print("### Multiple Regression Results (Y ~ X1 + X2 + X3) ###")
    print(model_all.summary())

### Multiple Regression Results (Y ~ X1 + X2 + X3) ###
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 3.543e+04
Date:                Sun, 31 Aug 2025   Prob (F-statistic):               0.00
Time:                        18:13:35   Log-Likelihood:                -727.62
No. Observations:                1000   AIC:                             1463.
Df Residuals:                     996   BIC:                             1483.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [4]:
# 3. Answer Questions 1 & 3
if 'model_all' in locals():
    # --- Question 1: Coefficient of X1 ---
    coeff_x1_full = model_all.params['X1']
    print("\n--- Question 1: Coefficient of X1 ---")
    print(f"The coefficient for X1 in the multiple regression is {coeff_x1_full:.4f}.")

    # --- Question 3: Most Significant Coefficient ---
    t_stats = model_all.tvalues.drop('const')
    most_significant_var = t_stats.abs().idxmax()
    print("--- Question 3: Most Significant Coefficient ---")
    print(f"T-statistics for each variable:\n{t_stats}\n")
    print(f"The variable with the highest absolute t-statistic is '{most_significant_var}'.")


--- Question 1: Coefficient of X1 ---
The coefficient for X1 in the multiple regression is 1.0071.
--- Question 3: Most Significant Coefficient ---
T-statistics for each variable:
X1     60.984011
X2     53.283212
X3    196.645240
dtype: float64

The variable with the highest absolute t-statistic is 'X3'.


In [5]:
# 4. Answer Question 2 (Simple Regression Comparison)
if 'model_all' in locals():
    print("\n" + "="*80 + "\n")
    print("--- Question 2: Greatest Difference in Coefficients ---")
    
    diffs = {}
    
    # Loop through each variable to perform a simple regression.
    for var in ['X1', 'X2', 'X3']:
        # Prepare the data for the simple regression.
        X_simple = sm.add_constant(df_1_1[[var]])
        
        # Fit the simple OLS model.
        model_simple = sm.OLS(Y, X_simple).fit()
        coeff_simple = model_simple.params[var]

        # Get the corresponding coefficient from the multiple regression model.
        coeff_multiple = model_all.params[var]

        # Calculate the absolute difference between the two coefficients.
        diffs[var] = abs(coeff_multiple - coeff_simple)
        
        print(f"\nAnalysis for {var}:")
        print(f"  - Simple regression coefficient: {coeff_simple:.4f}")
        print(f"  - Multiple regression coefficient: {coeff_multiple:.4f}")
        print(f"  - Absolute difference: {diffs[var]:.4f}")

    greatest_diff_var = max(diffs, key=diffs.get)
    print(f"\nThe variable with the greatest difference is '{greatest_diff_var}'.")



--- Question 2: Greatest Difference in Coefficients ---

Analysis for X1:
  - Simple regression coefficient: 1.8418
  - Multiple regression coefficient: 1.0071
  - Absolute difference: 0.8346

Analysis for X2:
  - Simple regression coefficient: 4.0836
  - Multiple regression coefficient: 1.9646
  - Absolute difference: 2.1190

Analysis for X3:
  - Simple regression coefficient: 3.0970
  - Multiple regression coefficient: 2.9755
  - Absolute difference: 0.1216

The variable with the greatest difference is 'X2'.


### Homework 1.2 Q4 - Q7

For Question 4 and 5:


Use NearestNeighbors to match data based on variables Z, given the file homework_1.2.csv.
Pick the best match in X = 0 corresponding to each X = 1. Use the Z values to perform the match: a good match with X = 1 is the item whose Z value is closest to the given sample's Z value with X = 0. I suggest using sklearn's NearestNeighbors to do this, but there are many ways to do it.  


For questions 6 and 7:



Use NearestNeighbors to match data based on variables Z, given the file homework_1.2.csv. 
Try approach B: Pick all of the matches in X = 0 that are within a distance 0.2 of each X = 1. Duplicates are okay, in case a given sample with X = 0 is a good match for multiple items with X = 1. 

In [13]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

df_1_2=pd.read_csv("homework_1.2.csv")

# Split data
df0 = df_1_2[df_1_2["X"] == 0].reset_index(drop=True)  # controls
df1 = df_1_2[df_1_2["X"] == 1].reset_index(drop=True)  # treated

Z0 = df0[["Z"]].to_numpy()
Z1 = df1[["Z"]].to_numpy()

# --- Q4 & Q5: 1-NN with replacement ---
nn = NearestNeighbors(n_neighbors=1, metric="euclidean").fit(Z0)
dist, idx = nn.kneighbors(Z1, return_distance=True)

q4_farthest = dist.max()  # farthest match distance

y1_mean = df1["Y"].mean()
y0_matched_mean = df0["Y"].to_numpy()[idx.ravel()].mean()
q5_effect = y1_mean - y0_matched_mean  # ATT-style sign

print("Q4 farthest distance:", q4_farthest)
print("Q5 effect (Y1 - matched Y0):", q5_effect)

# --- Q6 & Q7: radius=0.2, duplicates allowed ---
radius = 0.2
nnr = NearestNeighbors(radius=radius, metric="euclidean").fit(Z0)
neighbors_idx = nnr.radius_neighbors(Z1, return_distance=False)

# Q6 duplicates = total matches - unique controls
all_hits = np.concatenate([ix for ix in neighbors_idx if len(ix) > 0]) if len(neighbors_idx) else np.array([], dtype=int)
duplicates = all_hits.size - np.unique(all_hits).size
print("Q6 duplicates:", duplicates)

# Q7 effect: mean of group means
group_means = []
for ix in neighbors_idx:
    if len(ix) == 0:
        # Option A (what I did): skip empty groups
        # continue
        # Option B (fallback): use single nearest neighbor when empty
        # nearest_idx = nn.kneighbors(Z1[[k]], return_distance=False)[0,0]
        # group_means.append(df0["Y"].to_numpy()[nearest_idx])
        continue
    group_means.append(df0["Y"].to_numpy()[ix].mean())

q7_effect = y1_mean - np.mean(group_means)
print("Q7 effect (Y1 - mean of group means):", q7_effect)


Q4 farthest distance: 0.2102170871093757
Q5 effect (Y1 - matched Y0): 0.5433600651913855
Q6 duplicates: 685
Q7 effect (Y1 - mean of group means): 0.5844124774008066
